# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import classification_report, accuracy_score

import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///ETLproject.db')
df = pd.read_sql_table('ETLproject', engine)
X = df['message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [4]:
def tokenize(text):
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_test, X_train, y_test, y_train = train_test_split(X,y, random_state=42)
pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
print(classification_report(y_test, y_pred, target_names = y.columns))

                        precision    recall  f1-score   support

               related       0.84      0.91      0.87     14968
               request       0.78      0.41      0.54      3370
                 offer       0.00      0.00      0.00        92
           aid_related       0.72      0.60      0.65      8146
          medical_help       0.52      0.06      0.11      1542
      medical_products       0.52      0.05      0.08       966
     search_and_rescue       0.64      0.07      0.12       532
              security       0.50      0.00      0.01       338
              military       0.61      0.04      0.08       635
           child_alone       0.00      0.00      0.00         0
                 water       0.88      0.19      0.31      1242
                  food       0.80      0.54      0.64      2192
               shelter       0.83      0.25      0.38      1716
              clothing       0.70      0.05      0.10       304
                 money       0.89      

In [8]:
print(accuracy_score(y_test.values, y_pred))

0.228523128938


In [9]:
accuracy_score(y_test.values.reshape(-1,1), y_pred.reshape(-1,1))

0.94277387884272779

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f19222f9f28>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [11]:
parameters = {
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__max_depth':[20, 50]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2, verbose = 3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv.fit(X_train, y_train)
cv_y_pred = cv.predict(X_test)

Fitting 2 folds for each of 6 candidates, totalling 12 fits
[CV] clf__estimator__max_depth=20, clf__estimator__n_estimators=50 ...
[CV]  clf__estimator__max_depth=20, clf__estimator__n_estimators=50, score=0.18254456054087279, total=  11.9s
[CV] clf__estimator__max_depth=20, clf__estimator__n_estimators=50 ...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.9s remaining:    0.0s


[CV]  clf__estimator__max_depth=20, clf__estimator__n_estimators=50, score=0.17798954810943743, total=  12.0s
[CV] clf__estimator__max_depth=20, clf__estimator__n_estimators=100 ..


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   29.9s remaining:    0.0s


[CV]  clf__estimator__max_depth=20, clf__estimator__n_estimators=100, score=0.17977873386601106, total=  19.6s
[CV] clf__estimator__max_depth=20, clf__estimator__n_estimators=100 ..
[CV]  clf__estimator__max_depth=20, clf__estimator__n_estimators=100, score=0.18229326775284352, total=  19.5s
[CV] clf__estimator__max_depth=20, clf__estimator__n_estimators=200 ..
[CV]  clf__estimator__max_depth=20, clf__estimator__n_estimators=200, score=0.18315918869084205, total=  34.7s
[CV] clf__estimator__max_depth=20, clf__estimator__n_estimators=200 ..
[CV]  clf__estimator__max_depth=20, clf__estimator__n_estimators=200, score=0.1786043652013526, total=  34.4s
[CV] clf__estimator__max_depth=50, clf__estimator__n_estimators=50 ...
[CV]  clf__estimator__max_depth=50, clf__estimator__n_estimators=50, score=0.19514443761524278, total=  18.2s
[CV] clf__estimator__max_depth=50, clf__estimator__n_estimators=50 ...
[CV]  clf__estimator__max_depth=50, clf__estimator__n_estimators=50, score=0.194589609591146

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  6.8min finished


In [13]:
print(classification_report(y_test, cv_y_pred, target_names = y.columns))

                        precision    recall  f1-score   support

               related       0.79      0.99      0.88     14968
               request       0.91      0.25      0.39      3370
                 offer       0.00      0.00      0.00        92
           aid_related       0.78      0.56      0.65      8146
          medical_help       0.50      0.01      0.01      1542
      medical_products       0.82      0.03      0.06       966
     search_and_rescue       1.00      0.00      0.00       532
              security       0.00      0.00      0.00       338
              military       0.83      0.03      0.06       635
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.02      0.03      1242
                  food       0.91      0.16      0.28      2192
               shelter       0.92      0.08      0.15      1716
              clothing       0.67      0.04      0.07       304
                 money       1.00      

In [14]:
print(accuracy_score(y_test.values, cv_y_pred))

0.196147738333


In [15]:
print(accuracy_score(y_test.values.reshape(-1,1), cv_y_pred.reshape(-1,1)))

0.938664344381


In [16]:
print(cv.best_params_)

{'clf__estimator__max_depth': 50, 'clf__estimator__n_estimators': 50}


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    
    def starting_verb(self, text):
        sent_list = nltk.sent_tokenize(text)
        for sentence in sent_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            if len(pos_tags) > 1:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            else:
                return False
        return False
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [18]:
ab_pipeline = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer = tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        
        ('starting_verb', StartingVerbExtractor()),
    ])),
        
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [19]:
ab_pipeline.fit(X_train, y_train)

ab_y_pred = ab_pipeline.predict(X_test)

In [20]:
print(classification_report(y_test, ab_y_pred, target_names = y.columns))

                        precision    recall  f1-score   support

               related       0.81      0.96      0.87     14968
               request       0.71      0.52      0.60      3370
                 offer       0.03      0.01      0.02        92
           aid_related       0.74      0.61      0.67      8146
          medical_help       0.54      0.29      0.37      1542
      medical_products       0.56      0.31      0.40       966
     search_and_rescue       0.40      0.14      0.21       532
              security       0.14      0.04      0.06       338
              military       0.54      0.28      0.37       635
           child_alone       0.00      0.00      0.00         0
                 water       0.73      0.56      0.64      1242
                  food       0.80      0.57      0.66      2192
               shelter       0.72      0.50      0.59      1716
              clothing       0.59      0.34      0.43       304
                 money       0.51      

In [21]:
accuracy_score(y_test.values.reshape(-1,1), ab_y_pred.reshape(-1,1))

0.94434483661469981

In [22]:
ab_pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_... smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('starting_verb', StartingVerbExtractor())],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))], 'features': FeatureUnion(n_jobs=1,
        transformer_list=[('text_pipeline', Pipeline(memory=None,
      steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
    

In [23]:
ab_parameters = {
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__learning_rate':[0.001, 0.01, 0.1, 0.2, 0.5]
}

ab_cv = GridSearchCV(ab_pipeline, param_grid=ab_parameters, cv=2, verbose = 3)

In [24]:
ab_cv.fit(X_train, y_train)
ab_cv_y_pred = ab_cv.predict(X_test)

Fitting 2 folds for each of 15 candidates, totalling 30 fits
[CV] clf__estimator__learning_rate=0.001, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.001, clf__estimator__n_estimators=50, score=0.16994468346650277, total=  35.5s
[CV] clf__estimator__learning_rate=0.001, clf__estimator__n_estimators=50 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   45.6s remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.001, clf__estimator__n_estimators=50, score=0.1838303104826314, total=  35.4s
[CV] clf__estimator__learning_rate=0.001, clf__estimator__n_estimators=100 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.5min remaining:    0.0s


[CV]  clf__estimator__learning_rate=0.001, clf__estimator__n_estimators=100, score=0.17332513829133375, total=  52.7s
[CV] clf__estimator__learning_rate=0.001, clf__estimator__n_estimators=100 
[CV]  clf__estimator__learning_rate=0.001, clf__estimator__n_estimators=100, score=0.18444512757454656, total=  52.7s
[CV] clf__estimator__learning_rate=0.001, clf__estimator__n_estimators=200 
[CV]  clf__estimator__learning_rate=0.001, clf__estimator__n_estimators=200, score=0.17332513829133375, total= 1.4min
[CV] clf__estimator__learning_rate=0.001, clf__estimator__n_estimators=200 
[CV]  clf__estimator__learning_rate=0.001, clf__estimator__n_estimators=200, score=0.1869043959422072, total= 1.4min
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50, score=0.18715427166564227, total=  35.3s
[CV] clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50 
[CV]  clf__estimator__learning_rate=0

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 34.9min finished


In [25]:
print(classification_report(y_test, ab_cv_y_pred, target_names = y.columns))

                        precision    recall  f1-score   support

               related       0.79      0.98      0.88     14968
               request       0.81      0.41      0.54      3370
                 offer       0.22      0.02      0.04        92
           aid_related       0.78      0.51      0.62      8146
          medical_help       0.59      0.12      0.20      1542
      medical_products       0.76      0.17      0.28       966
     search_and_rescue       0.66      0.13      0.22       532
              security       0.42      0.01      0.03       338
              military       0.65      0.12      0.20       635
           child_alone       0.00      0.00      0.00         0
                 water       0.78      0.52      0.62      1242
                  food       0.78      0.68      0.73      2192
               shelter       0.83      0.46      0.59      1716
              clothing       0.77      0.33      0.46       304
                 money       0.72      

In [26]:
print(accuracy_score(y_test.values.reshape(-1,1), ab_cv_y_pred.reshape(-1,1)))

0.945494595564


In [27]:
print(ab_cv.best_params_)

{'clf__estimator__learning_rate': 0.1, 'clf__estimator__n_estimators': 200}


In [28]:
best_model = ab_cv.best_estimator_
print(best_model)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...ator=None,
          learning_rate=0.1, n_estimators=200, random_state=None),
           n_jobs=1))])


### 9. Export your model as a pickle file

In [29]:
filename = 'bestmodel.pkl'
pickle.dump(best_model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.